In [ ]:
import os
from pathlib import Path

from src.projects.fagradalsfjall.common.project_settings import (
    TS_PRIMARY_METRIC_DISPLAY_NAME,
    PATH_BLOG_POSTS,
)
from src.projects.fagradalsfjall.common.model_eval_plot_curves import generate_all_rmse_plots
from src.projects.fagradalsfjall.common.model_eval_plot_simulations import plot_simulations
from src.projects.fagradalsfjall.common.model_repo import ModelRepo
from src.projects.fagradalsfjall.common.model_eval import ModelEvalResult, ValidationType
from src.projects.fagradalsfjall.common.paths import get_blog_post_subfolder

from src.tools.datetime import format_timedelta

# 2. Visualization

In [ ]:
# -------------------------------------------------------------------------
#  Output path settings
# -------------------------------------------------------------------------
path_results_ols = get_blog_post_subfolder(5, "results_1_ols")
path_results_pls = get_blog_post_subfolder(5, "results_2_pls")
path_results_final = get_blog_post_subfolder(5, "results_3_final")

In [ ]:
# -------------------------------------------------------------------------
#  Load data
# -------------------------------------------------------------------------

# --- load models -------------------------------------
prev_model_eval_results = ModelRepo.load_models(["naive-mean", "naive-constant", "naive-exp-decay", "linear-ar"])
ols_model_eval_results = ModelRepo.load_models(["linear-ols"])
pls_model_eval_results = ModelRepo.load_models(["linear-pls"])

# --- post-processing ---------------------------------
model_ols = ols_model_eval_results["linear-ols"].ts_model
model_pls = pls_model_eval_results["linear-pls"].ts_model

In [ ]:
# -------------------------------------------------------------------------
#  PLOTS - OLS - Cross-Validation
# -------------------------------------------------------------------------

# --- 2D plots ----------------------------------------
alpha_values = model_ols.cv.results.all_param_values()["alpha"]
for alpha in alpha_values:
    fig, ax = model_ols.cv.results.plot_2d(x_param="p", y_param="n", param_filter=dict(alpha=alpha)).create(w=12, h=8)
    fig.suptitle(f"Cross-validation results - 'linear-ols' - [alpha={alpha}]")

    wd_str = str(alpha).replace(".", "_")
    fig.savefig(path_results_ols / f"cv_2d_pn_alpha_{wd_str}.png", dpi=600)

# --- 1D plots --- cross-section --------------------------
best_result = model_ols.cv.results.best_result.params

# settings
param_names = ["p", "n", "alpha"]

# create all 3 plots
for sweep_param in param_names:

    # construct param filter & title
    param_filter = {param_name: best_result[param_name] for param_name in param_names if param_name != sweep_param}
    title_postfix = ",".join(
        [f"{param_name}={best_result[param_name]}" for param_name in param_names if param_name != sweep_param]
    )

    # actual plot
    fig, ax = (
        model_ols.cv.results.plot_1d(
            param_names=sweep_param,
            param_filter=param_filter,
        )
        .set_fig_title(f"Cross-validation - 'linear-ols' - [{title_postfix}]")
        .set_y_label(TS_PRIMARY_METRIC_DISPLAY_NAME)
        .create(w=12, h=8)
    )
    fig.savefig(path_results_ols / f"cv_1d_cross_section_{sweep_param}.png", dpi=600)

# --- 1D plots --- best result ----------------------------
for param_name in param_names:

    fig, ax = (
        model_ols.cv.results.plot_1d(
            param_names=param_name,
        )
        .set_fig_title(f"Cross-validation - 'linear-ols' - best results for each value of '{param_name}'")
        .set_y_label(TS_PRIMARY_METRIC_DISPLAY_NAME)
        .set_x_label(param_name)
        .create(w=12, h=8)
    )
    fig.savefig(path_results_ols / f"cv_1d_best_{param_name}.png", dpi=600)

In [ ]:
# -------------------------------------------------------------------------
#  PLOTS - OLS - Simulations
# -------------------------------------------------------------------------

# RMSE curves
model_eval_results = prev_model_eval_results | ols_model_eval_results
generate_all_rmse_plots(model_eval_results, path_results_ols, highlight_models=["linear-ols"])

# simulations
plot_simulations(ols_model_eval_results, path_results_ols)

In [ ]:
# -------------------------------------------------------------------------
#  PLOTS - PLS - Cross-Validation
# -------------------------------------------------------------------------

# --- 2D plots ----------------------------------------
n_values = model_pls.cv.results.all_param_values()["n"]

for n in n_values:

    fig, ax = (
        model_pls.cv.results.plot_2d(x_param="p", y_param="n_components", param_filter={"n": n})
        .with_y_label("n_pls")
        .create(w=12, h=8)
    )
    fig.suptitle(f"Cross-validation results - 'linear-pls' - [n={n}]")

    fig.savefig(path_results_pls / f"cv_2d_p_npls_n_{n}.png", dpi=600)


# --- 1D plots --- best -----------------------------------
for param_name, display_name in [("p", "p"), ("n", "n"), ("n_components", "n_pls")]:

    fig, ax = (
        model_pls.cv.results.plot_1d(
            param_names=param_name,
        )
        .set_fig_title(f"Cross-validation - 'linear-pls' - best results for each value of '{display_name}'")
        .set_x_label(display_name)
        .set_y_label(TS_PRIMARY_METRIC_DISPLAY_NAME)
        .create(w=12, h=8)
    )
    fig.savefig(path_results_pls / f"cv_1d_best_{display_name}.png", dpi=600)

In [ ]:
# -------------------------------------------------------------------------
#  PLOTS - PLS - Simulations
# -------------------------------------------------------------------------

# RMSE curves
model_eval_results = prev_model_eval_results | pls_model_eval_results
generate_all_rmse_plots(model_eval_results, path_results_pls, highlight_models=["linear-pls"])

# simulations
plot_simulations(pls_model_eval_results, path_results_pls)

In [ ]:
# -------------------------------------------------------------------------
#  PLOTS - FINAL
# -------------------------------------------------------------------------

# RMSE curves
all_model_eval_results = prev_model_eval_results | ols_model_eval_results | pls_model_eval_results
generate_all_rmse_plots(all_model_eval_results, path_results_final, highlight_models=["linear-ols", "linear-pls"])

# simulations
plot_simulations(all_model_eval_results, path_results_final)

In [ ]:
df_metrics = ModelEvalResult.all_metric_values_as_df(all_model_eval_results, ValidationType.TEST)

df_metrics["max_accurate_lead_time_samples"] = df_metrics["max_accurate_lead_time"].copy()
df_metrics["max_accurate_lead_time"] = df_metrics["max_accurate_lead_time"].apply(
    lambda x: format_timedelta(x * 15 * 60)
)

display(df_metrics)